In [1]:
conda install -c conda-forge dcm2niix

Retrieving notices: ...working... done
^C

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [1]:
from nipype.interfaces.dcm2nii import Dcm2niix
import nipype.interfaces.io as nio
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
subjsource = nio.DataGrabber()

In [ ]:
subjsource

In [9]:
subjsource.inputs.base_directory = '/mnt/c/Users/since/Desktop/KdramaMay/dicom' # make sure to change output directory below
subjsource.inputs.template = 'S3[0-3]'
#subjsource.inputs.template = 'S29'
subjsource.inputs.sort_filelist = True
subjresults = subjsource.run()

converter = Dcm2niix()

if isinstance(subjresults.outputs.outfiles,str):
    subjresults.outputs.outfiles = [subjresults.outputs.outfiles]

In [10]:
converter

In [11]:
subjresults.outputs.outfiles

['/mnt/c/Users/since/Desktop/KdramaMay/dicom/S30',
 '/mnt/c/Users/since/Desktop/KdramaMay/dicom/S31',
 '/mnt/c/Users/since/Desktop/KdramaMay/dicom/S32',
 '/mnt/c/Users/since/Desktop/KdramaMay/dicom/S33']

In [12]:
for cur_subj in subjresults.outputs.outfiles:
    cur_len = len(cur_subj)
    cur_sid = 'sub-' + cur_subj[cur_len-2:cur_len]
    res_dir = cur_subj[0:cur_len-9]+'data/'
    print(res_dir)
    print('%s' %cur_sid)
    
    #for cur_type in ['*ENCODING*','*RECALL*','T2*','T1*','GRE*','PA*']:
    for cur_type in ['*_Encoding-*', '*_Recall-*', '*_T1_*', '*_gre_field_mapping_*', '*_PA_*']:       
        if cur_type == '*_T1_*': #or cur_type == 'T2*':
            cur_subdir = '/anat'
        elif cur_type == '*_gre_field_mapping_*' or cur_type == '*_PA_*':
            cur_subdir = '/fmap'
        else: 
            cur_subdir = '/func'
            
        #Grab Relevant Data
        datasource = nio.DataGrabber()
        datasource.inputs.base_directory = cur_subj
        datasource.inputs.template = cur_type
        datasource.inputs.sort_filelist = True
        datasource.inputs.raise_on_empty = False
        results = datasource.run()
        
        idx = 1
        if results.outputs.outfiles is not None:
            if isinstance(results.outputs.outfiles,str):
                results.outputs.outfiles = [results.outputs.outfiles]

        for cur_file in results.outputs.outfiles:
                print('cur_file is', cur_file)
                
                # set output file name
                if cur_type == '*_T1_*':
                    cur_output_filename = cur_sid + '_T1w'
                #elif cur_type == 'T2*':
                #    cur_output_filename = cur_sid + '_T2w'
                elif cur_type == '*_gre_field_mapping_*':
                    cur_output_filename = cur_sid + '_fmap'
                elif cur_type == '*_Encoding-*':
                    cur_output_filename = cur_sid + '_task-encoding_run-' + str(idx) + '_bold'
                elif cur_type == '*_Recall-*':
                    cur_output_filename = cur_sid + '_task-recall_run-' + str(idx) + '_bold'
                elif cur_type == '*_PA_*':
                    cur_output_filename = cur_sid + '_dir-pa' + '_epi'
                              
                converter.inputs.source_dir = cur_file
                converter.inputs.compression = 5
                converter.inputs.bids_format = True
                converter.inputs.output_dir = res_dir + cur_sid + cur_subdir 
                converter.inputs.out_filename = cur_output_filename
                converter.run()
                idx = idx + 1

                print(converter.inputs.output_dir + '/' + converter.inputs.out_filename)

/mnt/c/Users/since/Desktop/KdramaMay/data/
sub-30
cur_file is /mnt/c/Users/since/Desktop/KdramaMay/dicom/S30/series_7_Encoding-1
240207-16:41:15,454 nipype.interface INFO:
	 stdout 2024-02-07T16:41:15.454546:Chris Rorden's dcm2niiX version v1.0.20220720  GCC10.4.0 x86-64 (64-bit Linux)
240207-16:41:15,455 nipype.interface INFO:
	 stdout 2024-02-07T16:41:15.454546:Found 477 DICOM file(s)
240207-16:41:15,455 nipype.interface INFO:
	 stdout 2024-02-07T16:41:15.454546:Convert 477 DICOM as /mnt/c/Users/since/Desktop/KdramaMay/data/sub-30/func/sub-30_task-encoding_run-1_bold (80x80x48x477)
240207-16:41:16,205 nipype.interface INFO:
	 stdout 2024-02-07T16:41:16.205396:Compress: "/usr/bin/pigz" -b 960 -n -f -5 "/mnt/c/Users/since/Desktop/KdramaMay/data/sub-30/func/sub-30_task-encoding_run-1_bold.nii"
240207-16:41:16,205 nipype.interface INFO:
	 stdout 2024-02-07T16:41:16.205396:Conversion required 7.381395 seconds (1.950254 for core code).
/mnt/c/Users/since/Desktop/KdramaMay/data/sub-30/func/

## Protocol check용 .. 

In [ ]:
onehalf = ['0', '2', '2.5', '3', '3.5']
one = ['0', '3', '3.5']
two = ['0', '2', '2.5', '3', '3.5']
for cur_subj in subjresults.outputs.outfiles:
    cur_len = len(cur_subj)
    cur_sid = 'sub-' + cur_subj[cur_len-3:cur_len]
    res_dir = cur_subj[0:cur_len-10]+'data/'
    print('%s' %cur_sid)   
    
    for cur_type in ['1TR*','2TR*','1_5TR*']:
        if cur_type == '1TR*' or cur_type == '2TR*' or cur_type == '1_5TR*' :
            cur_subdir = '/func'

        datasource = nio.DataGrabber()
        datasource.inputs.base_directory = cur_subj
        datasource.inputs.template = cur_type
        datasource.inputs.sort_filelist = True
        datasource.inputs.raise_on_empty = False
        results = datasource.run()

        
        idx = 1
        if results.outputs.outfiles is not None:
            if isinstance(results.outputs.outfiles,str):
                results.outputs.outfiles = [results.outputs.outfiles]

            for cur_file in results.outputs.outfiles:
                print(cur_file)
                if  cur_type == '1TR*':
                    cur_output_filename = cur_sid + '_TR-' + str(cur_type[0]) + '_iso-' + one[idx] +  '_bold'
                    print(idx)
                elif cur_type == '2TR*':
                    cur_output_filename = cur_sid + '_TR-' + str(cur_type[0]) + '_iso-'  + two[(idx)] + '_bold'
                    print(idx)
                elif cur_type == '1_5TR*':
                    cur_output_filename = cur_sid + '_TR-' + str(cur_type[0:3]) + '_iso-'  + onehalf[(idx)] + '_bold'
                              
                converter.inputs.source_dir = cur_file
                converter.inputs.compression = 5
                converter.inputs.bids_format = True
                converter.inputs.output_dir = res_dir + cur_sid + cur_subdir 
                converter.inputs.out_filename = cur_output_filename
                converter.run()
                idx = idx + 1

                print(converter.inputs.output_dir + '/' + converter.inputs.out_filename)

# 디렉토리 체크

In [28]:
for cur_subj in subjresults.outputs.outfiles:
    cur_len = len(cur_subj)
    cur_sid = 'sub-' + cur_subj[cur_len-2:cur_len]
    print('%s' %cur_sid)

sub-01
sub-02
sub-03
sub-04
sub-05
